<a href="https://colab.research.google.com/github/anitayadav3/EmotionRecognitionInConversation/blob/master/ALBERT_XLarge_implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install tensorflow_text
import tensorflow_text as text 

     |████████████████████████████████| 3.4MB 17.6MB/s 


In [19]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
from nltk.tokenize import word_tokenize
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input, GRU, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow_hub as hub
import pickle
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from keras.utils.np_utils import to_categorical  
import string
import requests

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [6]:
preprocessor = hub.KerasLayer("https://hub.tensorflow.google.cn/tensorflow/albert_en_preprocess/2")
encoder = hub.KerasLayer("https://hub.tensorflow.google.cn/tensorflow/albert_en_xlarge/2",trainable=True)

In [7]:
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string)
encoder_inputs = preprocessor(text_input)
outputs = encoder(encoder_inputs)
pooled_output = outputs["pooled_output"]      # [batch_size, 2048].
out = Dense(100, activation='relu')(pooled_output)
out = Dense(100, activation='relu')(out)
out = Dense(6, activation='softmax')(out)

In [8]:
L=[]
model = tf.keras.Model(text_input, out)
sentences = tf.constant(["My text"])
print((model(sentences)))

tf.Tensor([[0.5728689  0.10440885 0.10842568 0.06997757 0.0524542  0.09186476]], shape=(1, 6), dtype=float32)


In [11]:
def build_model(bert_layer):
  text_input = tf.keras.layers.Input(shape=(), dtype=tf.string)
  encoder_inputs = preprocessor(text_input)
  outputs = bert_layer(encoder_inputs)
  pooled_output = outputs["pooled_output"]      # [batch_size, 2048].
  out = Dense(100, activation='relu')(pooled_output)
  out = Dense(100, activation='relu')(out)
  out = Dense(6, activation='softmax')(out)
  model = Model(inputs=text_input, outputs=out)
  model.compile(Adam(lr=2e-6), loss='categorical_crossentropy', metrics=['accuracy'])
  return model

In [12]:
model = build_model(encoder)
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None,)]            0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        {'input_word_ids': ( 0           input_3[0][0]                    
__________________________________________________________________________________________________
keras_layer_1 (KerasLayer)      {'encoder_outputs':  58724864    keras_layer[2][0]                
                                                                 keras_layer[2][1]                
                                                                 keras_layer[2][2]                
____________________________________________________________________________________________

In [13]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [14]:
with open('/content/gdrive/My Drive/iemocap/train/sentences.pkl', 'rb') as f:
    data = pickle.load(f)
with open('/content/gdrive/My Drive/iemocap/train/labels.pkl', 'rb') as f:
    labels = pickle.load(f)
with open('/content/gdrive/My Drive/iemocap/test/sentences.pkl', 'rb') as f:
    test_data = pickle.load(f)
with open('/content/gdrive/My Drive/iemocap/test/labels.pkl', 'rb') as f:
    test_labels = pickle.load(f)
with open('/content/gdrive/MyDrive/iemocap/train/conversation_length.pkl', 'rb') as f:
    train_convlen = pickle.load(f)
with open('/content/gdrive/MyDrive/iemocap/test/conversation_length.pkl', 'rb') as f:
    test_convlen = pickle.load(f)

In [15]:
def preprocessing(data,labels):
  processed_data=[]
  processed_label=[]
  for i in range(0,len(data)):
    for j in range(0,len(data[i])):
      intermediate_data=[]
      intermediate_label=[]
      for k in range(0,len(data[i][j])):
        text=data[i][j][k]
        if text != '<eos>'and text!='<pad>':
          intermediate_data.append(text)
      processed_data.append(intermediate_data)
  for i in labels:
    for j in i:
      processed_label.append(j)
  return processed_data,processed_label

In [16]:
processed_data,processed_label = preprocessing(data,labels)
test_processed_data,test_processed_label = preprocessing(test_data,test_labels)

In [17]:
for i in range(0,len(processed_data)):
  processed_data[i]= ' '.join(processed_data[i])
for i in range(0,len(test_processed_data)):
  test_processed_data[i]=' '.join(test_processed_data[i])

In [20]:
processed_data=np.asarray(processed_data)
test_processed_data=np.asarray(test_processed_data)
Y=to_categorical(processed_label, num_classes=6)
test_Y=to_categorical(test_processed_label, num_classes=6)

In [21]:
train_input = processed_data
test_input = test_processed_data
train_labels = Y
test_labels = test_Y

In [ ]:
train_history = model.fit(
    train_input, train_labels,
    epochs=6,
    batch_size=1
)

Epoch 1/6
1067/4699 [=====>........................] - ETA: 23:47 - loss: 1.7617 - accuracy: 0.2212

In [ ]:
y_pred=model.predict(test_input, batch_size=1)

In [ ]:
y_pred1=np.argmax(y_pred,axis=1)
test_processed_label=np.asarray(test_processed_label)
print("Accuracy : " + str(accuracy_score(test_processed_label, y_pred1)))
print("Weighted F1-score : " + str(f1_score(test_processed_label, y_pred1, average='weighted')))